### Лекция 5. Шаблоны

<br />

##### Какая идея стоит за шаблонами

Ранее мы познакомились с возможностью перегрузки функций. Давайте вспомним её на примере swap:

```c++
// поменять местами два int
void my_swap(int& a, int& b)
{
    int tmp = a;
    a = b;
    b = tmp;    
}

// поменять местами два short
void my_swap(short& a, short& b)
{
    short tmp = a;
    a = b;
    b = tmp;
}

// поменять местами два float
void my_swap(float& a, float& b)
{
    float tmp = a;
    a = b;
    b = tmp;
}

...
```

Вечер начинает быть томным ...

Для решения проблем дублирования кода придуманы шаблоны:

```c++
// напишем шаблон - как должна выглядеть функция
template<typename T>
void my_swap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}
```

Применение шаблона:

```c++
int a = 3, b = 5;

// вызов my_swap(int&, int&), тип T указывается программистом явно
my_swap<int>(a, b);

// вызов my_swap(int&, int&), тип T выводится компилятором автоматически
my_swap(a, b); 


float x = 3.f, y = 5.f;
my_swap(x, y);
my_swap<float>(x, y);
```

Важное свойство шаблонов по сравнению с перегрузкой функций:
* шаблон компилируется только тогда, когда он вызывается
* шаблон компилируется только для тех типов, для которых он вызывается.

_и в каждом cpp-файле шаблон компилируется снова и снова_

Показать пример на godbolt.org, позакомментировать функции, продемонстрировать разницу в выхлопе компилятора.

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Особенности шаблонов по сравнению с перегруженными функциями:
* компилируется только то, что инстанциируется в коде
* компилируется столько раз, в скольки единицах трансляции инстанциируется:
    * можно в одном cpp-файле 10 раз позвать myswap(int&, int&) - эта функция скомпилируется единожды
    * можно в 10 cpp-файлах один раз позвать myswap(int&, int&) - эта функция скомпилируется 10 раз
* накладные расходы во время компиляции на кодогенерацию при истанциации
* позволяет компилятору агрессивнее оптимизировать. Раскомментировать `__attribute__((noinline))` из примера и показать какой код сгенерирует компилятор. Объяснить, почему.
* позволяет нарушать ODR

Коротко:

* (+) меньше кода
* (+) быстрее
* (-) дольше компилируется
* (-) сложнее писать

__Вопросы__:
* Где поместить шаблонную функцию, которую нужно использовать в разных cpp-файлах?
* Где поместить её реализацию?
* Может ли шаблонная функция содержать некомпилирующийся код?

<br />

##### Специализация

Перегрузка функций позволяла сделать `myswap` у `std::string` более эффективно, без лишнего копирования памяти:

```c++
void myswap(int& a, int& b) { ... }
void myswap(short& a, short& b) { ... }

void myswap(std::string& a, std::string& b)
{
    a.swap(b);
}
```

Шаблоны тоже позволяют специализировать поведение функций, если наложить на шаблонный параметр ограничение, например:

(закинуть этот код на godbolt, показать во что компилируется программа)

```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

template<>
void __attribute__ ((noinline)) myswap<std::string>(std::string& a, std::string& b)
{
    a.swap(b);
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Во-первых, шаблон может иметь несколько параметров, а во-вторых, параметры не обязаны быть типами. Они могут быть, например, целыми числами:

```c++
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}

int a = add_value<5, int>(100);
int a = add_value<5>(100);

// 1. шаблон специализирован программистом частично, тип Т компилятор определит сам
// 2. параметром шаблона выступает целое число.
```

__Вопрос__: Какую информацию здесь компилятор использует, чтобы вывести тип `T`?

<br />

##### Шаблонные классы

Аналогично функциям, классы тоже могут быть шаблонными:

Пример структуры:

```c++
// N-мерный вектор из курса линейной алгебры типа T
template<typename T, int N>
struct VectorN
{
    T data[N];
};

// в качестве примера запишем операции сложения и умножения для таких векторов

template<typename T, int N>
VectorN<T, N> operator +(const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    VectorN<T, N> rv;
    for (int i = 0; i < N; ++i)
        rv.data[i] = l.data[i] + r.data[i];
    return rv;    
}

template<typename T, int N>
T operator * (const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    T rv = 0;
    for (int i = 0; i < N; ++i)
        rv += l.data[i] * r.data[i];
    return rv;
}
```

Пример шаблонного класса:

Напишем свой собственный `optional`. Он будет не так хорош, как `std::optional`, потому что мы не знаем пока всех необходимых трюков С++, но для начала пойдёт.

Идея `optional` - класс-обёртка над значением, которое может отсутствовать.

Для начала объявим шаблонный класс:

```c++
template<typename T>
class Optional {
    ...
};
```

Добавим в класс поля:

```c++
template<typename T>
class Optional {
    
private:
    T value_;
    bool has_value_;
    
    ...
};
```

**Вопрос:** Мы выбрали способ хранения объекта как поле класса, потому что так проще для демонстрации. Какие у него недостатки? Как их обойти?

<details>
<summary>Ответ</summary>

* Недостаток - объект существует даже когда он не нужен
* Вариант обхода 1 - хранить объект в куче через new. Проблема: расходы на new/delete.
* Вариант обхода 2 - кусок сырой памяти под объект + placement new и ручной вызов деструкторов (трюк в std::optional)

</details>

Добавим конструкторы:
    
```c++
template<typename T>
class Optional {
    ...
        
public:
    Optional()
        : has_value_(false)
    {}
        
    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}   
};

//
// usage:
//
Optional<std::string> maybe_string_1;
Optional<std::string> maybe_string_2("hello world");
```

**Вопрос:** Почему `const &` у `another_value` ?

**Вопрос:** Что мы должны срочно добавить в класс?

Срочно добавим:


```c++
template<typename T>
class Optional {
    ...

public:
    Optional(const Optional&) = default;
    Optional(Optional&&) = default;
    Optional& operator = (const Optional&) = default;
    Optional& operator = (Optional&&) = default;
    ~Optional() = default;
};

//
// usage:
//
Optional<std::string> maybe_string_1("hello world");
Optional<std::string> maybe_string_2 = maybe_string_1;
Optional<std::string> maybe_string_3 = std::move(maybe_string_1);
```

Добавим немного функционала:

```c++
template<typename T>
class Optional {
    ...
        
public:
    bool has_value() const { return has_value_; }
    
    T&       get_value()       { return value_; }
    const T& get_value() const { return value_; }
    
    T*       get_ptr()       { return has_value_ ? &value_ : nullptr; }    
    const T* get_ptr() const { return has_value_ ? &value_ : nullptr; }
    
    void reset() {
        value_ = T();
        has_value_ = false;
    }

    void reset(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }
    
    void emplace() {
        value_ = T();
        has_value_ = true;
    }
};

//
// usage:
//
Optional<std::string> maybe_string_1("hello world");

if (maybe_string_1.has_value())
    std::cout << maybe_string_1.get_value() << std::endl;

if (std::string* s = maybe_string_1.get_ptr())
    std::cout << *s << std::endl;

maybe_string_1.reset();

maybe_string_1.emplace();
```

Добавим операторы сравнения:
    
```c++
template<typename T>
class Optional { ... };

template<typename T>
bool operator == (const Optional<T>& lhs, const Optional<T>& rhs) {
    if (!lhs.has_value())
        return !rhs.has_value();
    
    if (!rhs.has_value())
        return false;
    
    return lhs.get_value() == rhs.get_value();    
}

template<typename T>
bool operator != (const Optional<T>& lhs, const Optional<T>& rhs) {
    return !(lhs == rhs);
}

template<typename T>
bool operator == (const Optional<T>& lhs, const T& rhs) {
    return
        lhs.has_value() &&
        lhs.get_value() == rhs;
}

template<typename T>
bool operator == (const T& lhs, const Optional<T>& rhs) {
    return rhs == lhs;
}

template<typename T>
bool operator != (const Optional<T>& lhs, const T& rhs) {
    return !(lhs == rhs);
}

template<typename T>
bool operator != (const T& lhs, const Optional<T>& rhs) {
    return !(lhs == rhs);
}



//
// usage:
//
Optional<std::string> maybe_string_1("hello world");
Optional<std::string> maybe_string_2 = maybe_string_1;

if (maybe_string_1 != maybe_string_2)
    std::cout << "unreachable!" << std::endl;

Optional<int> maybe_int;

if (maybe_string_1 == maybe_int)  // compile-time error!
    std::cout << "unreachable!" << std::endl;
```

**Вопрос:** какая проблема в этом коде?

```c++
Optional<std::string> maybe_string("hello world");

if (maybe_string == "C++ is designed for faster code")
    std::cout << "Fast enough?" << std::endl;
```

Решим эту проблему, добавим ещё один более хитрый оператор сравнения:
    
```c++

template<typename T, typename U>
bool operator == (const Optional<T>& lhs, const U& rhs) {
    return
        lhs.has_value() &&
        lhs.get_value() == rhs;
}

template<typename T, typename U>
bool operator == (const T& lhs, const Optional<U>& rhs) {
    return rhs == lhs;
}

template<typename T, typename U>
bool operator != (const Optional<T>& lhs, const U& rhs) {
    return !(lhs == rhs);
}

template<typename T, typename U>
bool operator != (const T& lhs, const Optional<U>& rhs) {
    return !(rhs == lhs);
}

//
// usage:
//

Optional<std::string> maybe_string_1("hello world");

if (maybe_string == "C++ is designed for faster code")
    std::cout << "Fast enough?" << std::endl;
```

Чтобы не было скучно, добавим в шаблонный класс шаблонный метод.


```c++
template<typename T>
class Optional {
    ...

public:
    // уже было
    void emplace(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }
    
    // добавили
    template<typename U>
    void emplace(const U& source) {
        value_ = T(source);
        has_value_ = true;
    }
};

//
// usage:
//
Optional<std::string> maybe_string;
maybe_string.emplace("hello world");
```

**Вопрос:** Что поменялось при вызове `emplace`?

Самое время добавить в класс шаблонный конструктор!

```c++
template<typename T>
class Optional {
    ...
        
public:
    // уже было
    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}

    // добавили
    template<typename U>
    Optional(const U& source)
        : value_(source)
        , has_value_(true)
    {}
};

//
// usage:
//
Optional<std::string> maybe_string("hello world");
```

**Вопрос:** Что поменялось в вызове конструктора от `const char*` ?

**Вопрос:** Можем ли мы что-нибудь выиграть от шаблонного деструктора ?

<br />

Пример целиком без операторов. Закинуть на godbolt, показать сколько каких методов генерируется, не забыть убрать оптимизации и demangle.

```c++
#include <string>

template<typename T>
class Optional {
private:
    T value_;
    bool has_value_;

public:
    Optional()
        : has_value_(false)
    {}

    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}

    template<typename U>
    Optional(const U& source)
        : value_(source)
        , has_value_(true)
    {}

    Optional(const Optional&) = default;
    Optional(Optional&&) = default;
    Optional& operator = (const Optional&) = default;
    Optional& operator = (Optional&&) = default;
    ~Optional() = default;

    bool has_value() const { return has_value_; }

    T&       get_value()       { return value_; }
    const T& get_value() const { return value_; }

    T*       get_ptr()       { return has_value_ ? &value_ : nullptr; }    
    const T* get_ptr() const { return has_value_ ? &value_ : nullptr; }

    void reset() {
        value_ = T();
        has_value_ = false;
    }

    void reset(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }

    void emplace() {
        value_ = T();
        has_value_ = true;
    }

    template<typename U>
    void emplace(const U& source) {
        value_ = T(source);
        has_value_ = true;
    }
};


std::string f() {
    Optional<std::string> opt;

    opt.emplace("hello");
    opt.emplace("C++");
    opt.emplace("world");        

    opt.emplace(std::string("-hello"));
    opt.emplace(std::string("-C++"));
    opt.emplace(std::string("-world"));

    return opt.get_value();
}
```

<br />

##### Специфика компиляции шаблонов

Создадим свой собственный тип, которому запретим copy assignment:

```c++
class C {
public:
    C() = default;
    C(const C&) = default;
    C(C&&) = default;
    C& operator = (const C&) = delete;  // DELETE
    C& operator = (C&&) = default;
    ~C() = default;
};

```

Сделаем от него `Optional`:

```c++
Optional<C> opt;
```

**Вопрос:** Какие-нибудь возникли подозрения?

<br />

Ответ: компилируются только те методы, что вызываются:
        
```c++
Optional<C> opt1;       // ok
Optional<C> opt2;       // ok
Optional<C> opt3(C());  // ok

if (opt1 == opt2)  // ok
    std::cout << "equal!" << std::endl;

if (opt1 != opt3)  // ok
    std::cout << "not equal!" << std::endl;

opt1.reset();  // ok

opt1 = opt3;       // compilation error: C::operator = (const C&) is deleted

opt1.reset(C());   // compilation error: C::operator = (const C&) is deleted
```

<br />

##### Member types

```c++
template<typename T>
class Optional
{
public:
    using value_type = T;

    ...
};

//
// usage:
//
typename Optional<T>::value_type value;
```

Практический смысл этой конструкции станет понятен, когда будете делать итерирование по полиномам во втором домашнем задании.

<br />

##### just for fun: compile-time факториал

Теперь мы разбираемся в шаблонах достаточно чтобы посчитать факториал во время компиляции на шаблонах (разобрать пример, показать результат в godbolt).

Примечание: C++ значительно эволюционировал, и больше во время компиляции таким образом вычисления не проводят. Пример исключительно ученический. Compile-time вычисления будут рассмотрены в курсе далее.

```c++
template<unsigned N>
struct Factorial
{
    static const int value = N * Factorial<N - 1>::value;
};

template<>
struct Factorial<0>
{
    static const int value = 1;
};

int main()
{
    return Factorial<10>::value;
}
```

__Вопросы__:
* Что делает следующий пример?

```c++
#include <cstdio>

template<unsigned N>
struct f
{
    static const int value = f<N-1>::value + f<N-2>::value;
};

template<>
struct f<0>
{
    static const int value = 0;
};

template<>
struct f<1>
{
    static const int value = 1;
};

int main()
{
    printf("%i\n", f<45>::value);
}
```

<br />

##### Примеры шаблонных функций / классов из стандартной библиотеки

Рассмотреть примеры подробнее, специализации (если есть). Показать секции member types, non-member functions etc.

https://en.cppreference.com/w/cpp/numeric/complex

https://en.cppreference.com/w/cpp/container/vector

<br />

За пределами лекции:
* Частичная специализация шаблонов (для классов)

Во второй части курса про шаблоны:
* SFINAE
* variadic templates
* type traits
* tag dispatching

<br />

**Замечания после лекции:**
    
* По традиции осталось минут 30 от лекции. Это время можно разбавить примерами или рассказать что-нибудь полезное.
* Переделать материал по member types. Сейчас сумбурно и не очевидно, зачем он нужен.